Authors: Alberto Padilla y Marco Hernani

This jupyter contains a emotion recognition model trained and tested with the dataset SEED GER signals

In [1]:
## Dataset proporcionado por (citar correctamente a los autores) ##

In [1]:
import pandas as pd
import numpy as np
import os
import mne

In [3]:
#path where data is located
data_path = "../SEED_FRA_OUTPUT"

In [4]:
def get_file_path(_path, _extension=''):
    """Gets the path for every file
    _path: the path where all the data is located
    _extension: the extension of the file
    returns the path as a string for every file with the wanted extension in
    the folder
    """
    file_names = os.listdir(_path)
    file_names = filter(lambda s: _extension in s, file_names)
    return [_path + "/" + name for name in file_names]
    

We get features from the signal readed in a receptor

In [47]:
from scipy.signal import hilbert

def extract_features(_signal):
    """Gets features of the signal using the aplitud envelope"""
    #amplitud_envelope = np.real(hilbert(_signal))
    analytic_signal = hilbert(_signal)
    amplitude_envelope = np.abs(analytic_signal)
    features = np.array( [np.mean(amplitude_envelope), np.std(amplitude_envelope), np.max(amplitude_envelope), np.min(amplitude_envelope)] )
    return features


In [16]:
def extract_features_2(_signal):
    """Gets features of the signal using the aplitud envelope"""
    
    data_1 = np.real(hilbert(_signal))
    data_2 = np.imag(hilbert(_signal))
  
    features = np.array( [np.mean(data_1), np.std(data_1), np.max(data_1), np.min(data_1), np.mean(data_2), np.std(data_2), np.max(data_2), np.min(data_2)] )
    
    return features

Extracting characteristics for only one signal

- Create a function to do this with all the signals

- Create the data for train our model

- :D

In [48]:
def get_trial_features(_eeg_data):
    """Gets the features for a trial
    _eeg_data: content from the fif files with the eeg information for a trial
    returns: 4 numbers (mean, std, max, min) those are the median of all those features
    in each signal"""
    trial_signals = _eeg_data.get_data()

    trial_features = []

    for signal in trial_signals:
        features = extract_features(signal)
        
        #features = extract_features_2(signal)
        trial_features.append(features)

    trial_features = np.array(trial_features)
    
    trial_resume = []
    for feature in trial_features.T:
        trial_resume.append( np.median(feature) ) 
    
    return trial_resume

In [92]:
def get_trial_resume(_eeg_data):
    """Gets the features for a trial
    _eeg_data: content from the fif files with the eeg information for a trial
    returns: 4 numbers (mean, std, max, min) those are the median of all those features
    in each signal"""
    trial_signals = _eeg_data.get_data()

    trial_features = []

    for signal in trial_signals:
        features = extract_features(signal)
        
        #features = extract_features_2(signal)
        trial_features.append(features)

    trial_features = np.array(trial_features)
    
    return trial_features

In [100]:
def get_data_DL(_path, _emotion):
    """Gets all the features for all the trials in a folder where each file is a trial
    _path: path of the folder
    emotion: the emotion label
    returns: a dataframe where the columns are (mean, std, max, min, emotion) and there is a row
    for each trial in the folder"""
    file_names = get_file_path(_path, _extension='.fif')
    
    list_2d = []
    
    for name in file_names:
        eeg_data = mne.io.read_raw_fif(name, preload=True)
        trial_resume = get_trial_resume(eeg_data)

        list_2d.append(trial_resume)
    
    return list_2d

In [49]:
def get_folder_data(_path, _emotion):
    """Gets all the features for all the trials in a folder where each file is a trial
    _path: path of the folder
    emotion: the emotion label
    returns: a dataframe where the columns are (mean, std, max, min, emotion) and there is a row
    for each trial in the folder"""
    file_names = get_file_path(_path, _extension='.fif')
    
    #df = pd.DataFrame( columns= ['mean_trial', 'std_trial', 'max_trial', 'min_trial', 'mean_trial_i', 'std_trial_i', 'max_trial_i', 'min_trial_i', 'emotion'], index=[i for i in range(len(file_names))])

    df = pd.DataFrame( columns= ['mean_trial', 'std_trial', 'max_trial', 'min_trial', 'emotion'], index=[i for i in range(len(file_names))])
    
    for i, name in enumerate(file_names):
        eeg_data = mne.io.read_raw_fif(name, preload=True)
        trial_resume = get_trial_features(eeg_data)

        trial_resume.append(_emotion)
        
        df.iloc[i] = trial_resume
    
    return df

In [9]:
###Extra: Comprobar que la frecuencia de muestreo es la misma para todos los ficheros con código
###Extra: Comprobar que una frecuencia de muestreo diferente no afecta cuando se resumen los datos de este modo


Cuando se añade en un array se crea un nuevo objeto en el proceso

-Usar funciones creadas

In [50]:
df_positive = get_folder_data("../SEED_FRA_OUTPUT/POSITIVE", 1)
df_negative = get_folder_data("../SEED_FRA_OUTPUT/NEGATIVE", -1)
df_neutral = get_folder_data("../SEED_FRA_OUTPUT/NEUTRAL", 0)

Opening raw data file ../SEED_FRA_OUTPUT/POSITIVE/1_1_trial_0_signals.fif...
    Range : 21000 ... 193000 =     21.000 ...   193.000 secs
Ready.
Reading 0 ... 172000  =      0.000 ...   172.000 secs...


C:\Users\marco\AppData\Local\Temp\ipykernel_6904\3667833851.py:14: RuntimeWarning: This filename (../SEED_FRA_OUTPUT/POSITIVE/1_1_trial_0_signals.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg_data = mne.io.read_raw_fif(name, preload=True)


Opening raw data file ../SEED_FRA_OUTPUT/POSITIVE/1_1_trial_11_signals.fif...
    Range : 2327000 ... 2471000 =   2327.000 ...  2471.000 secs
Ready.
Reading 0 ... 144000  =      0.000 ...   144.000 secs...


C:\Users\marco\AppData\Local\Temp\ipykernel_6904\3667833851.py:14: RuntimeWarning: This filename (../SEED_FRA_OUTPUT/POSITIVE/1_1_trial_11_signals.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg_data = mne.io.read_raw_fif(name, preload=True)


Opening raw data file ../SEED_FRA_OUTPUT/POSITIVE/1_1_trial_12_signals.fif...
    Range : 2492000 ... 2599000 =   2492.000 ...  2599.000 secs
Ready.
Reading 0 ... 107000  =      0.000 ...   107.000 secs...


...

C:\Users\marco\AppData\Local\Temp\ipykernel_6904\3667833851.py:14: RuntimeWarning: This filename (../SEED_FRA_OUTPUT/NEUTRAL/8_3_trial_2_signals.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg_data = mne.io.read_raw_fif(name, preload=True)


Opening raw data file ../SEED_FRA_OUTPUT/NEUTRAL/8_3_trial_5_signals.fif...
    Range : 1103000 ... 1232000 =   1103.000 ...  1232.000 secs
Ready.
Reading 0 ... 129000  =      0.000 ...   129.000 secs...


C:\Users\marco\AppData\Local\Temp\ipykernel_6904\3667833851.py:14: RuntimeWarning: This filename (../SEED_FRA_OUTPUT/NEUTRAL/8_3_trial_5_signals.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg_data = mne.io.read_raw_fif(name, preload=True)


Opening raw data file ../SEED_FRA_OUTPUT/NEUTRAL/8_3_trial_7_signals.fif...
    Range : 1572000 ... 1718000 =   1572.000 ...  1718.000 secs
Ready.
Reading 0 ... 146000  =      0.000 ...   146.000 secs...


C:\Users\marco\AppData\Local\Temp\ipykernel_6904\3667833851.py:14: RuntimeWarning: This filename (../SEED_FRA_OUTPUT/NEUTRAL/8_3_trial_7_signals.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg_data = mne.io.read_raw_fif(name, preload=True)


In [51]:
df_positive

,mean_trial,std_trial,max_trial,min_trial,emotion
0,0.000012,0.000007,0.000062,0.0,1
1,0.000012,0.000007,0.000064,0.0,1
2,0.000013,0.000008,0.000058,0.0,1
3,0.000013,0.000009,0.000094,0.0,1
4,0.000012,0.000007,0.000073,0.0,1
...,...,...,...,...,...
163,0.000009,0.000005,0.000075,0.0,1
164,0.000009,0.000006,0.000085,0.0,1
165,0.000009,0.000006,0.000094,0.0,1
166,0.000011,0.000008,0.000092,0.0,1


In [52]:
df_negative

,mean_trial,std_trial,max_trial,min_trial,emotion
0,0.000013,0.000008,0.000071,0.0,-1
1,0.000011,0.000007,0.000061,0.0,-1
2,0.000011,0.000007,0.000066,0.0,-1
3,0.000012,0.000008,0.000077,0.0,-1
4,0.000012,0.000008,0.000078,0.0,-1
...,...,...,...,...,...
163,0.000008,0.000005,0.000045,0.0,-1
164,0.000024,0.00002,0.000162,0.0,-1
165,0.00001,0.000006,0.000097,0.0,-1
166,0.00001,0.000007,0.000144,0.0,-1


In [53]:
df_neutral

,mean_trial,std_trial,max_trial,min_trial,emotion
0,0.000013,0.000009,0.000095,0.0,0
1,0.000013,0.000008,0.000076,0.0,0
2,0.000013,0.000009,0.000095,0.0,0
3,0.000014,0.000009,0.00008,0.0,0
4,0.000013,0.000008,0.000086,0.0,0
...,...,...,...,...,...
163,0.000009,0.000006,0.000098,0.0,0
164,0.000008,0.000005,0.000043,0.0,0
165,0.000012,0.000008,0.000098,0.0,0
166,0.000012,0.000009,0.000111,0.0,0


In [54]:
df_positive['mean_trial'][0]

1.194411942005399e-05

In [ ]:
features

In [55]:
df_total = pd.concat([df_positive, df_negative, df_neutral], ignore_index=True)
df_total

,mean_trial,std_trial,max_trial,min_trial,emotion
0,0.000012,0.000007,0.000062,0.0,1
1,0.000012,0.000007,0.000064,0.0,1
2,0.000013,0.000008,0.000058,0.0,1
3,0.000013,0.000009,0.000094,0.0,1
4,0.000012,0.000007,0.000073,0.0,1
...,...,...,...,...,...
499,0.000009,0.000006,0.000098,0.0,0
500,0.000008,0.000005,0.000043,0.0,0
501,0.000012,0.000008,0.000098,0.0,0
502,0.000012,0.000009,0.000111,0.0,0


In [56]:
#Guardando csv
df_total.to_csv('data_for_model.csv', index=False)

In [81]:
df_total = pd.read_csv('data_for_model_2.csv')

In [82]:
df_total.dtypes

mean_trial      float64
std_trial       float64
max_trial       float64
min_trial       float64
mean_trial_i    float64
std_trial_i     float64
max_trial_i     float64
min_trial_i     float64
emotion           int64
dtype: object

In [83]:
df_total = df_total.astype(float)

In [84]:
from sklearn.model_selection import train_test_split

X = df_total.drop(["emotion"], axis=1)
y = df_total.emotion

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [85]:
from sklearn.svm import SVC

model = SVC()
model.fit(X_train, y_train)

SVC()

In [86]:
accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.3564356435643564


In [87]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression()

In [89]:
accuracy = clf.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.297029702970297


In [90]:
from sklearn.naive_bayes import GaussianNB

gNB = GaussianNB()
gNB.fit(X_train, y_train)

GaussianNB()

In [91]:
accuracy = gNB.score(X_test, y_test)
print("Precisión del modelo: {:.2f}%".format(accuracy*100))

Precisión del modelo: 35.64%


In [93]:
eeg_data = mne.io.read_raw_fif("../SEED_FRA_OUTPUT/POSITIVE/1_1_trial_0_signals.fif", preload=True)

get_trial_resume(eeg_data)

Opening raw data file ../SEED_FRA_OUTPUT/POSITIVE/1_1_trial_0_signals.fif...
    Range : 21000 ... 193000 =     21.000 ...   193.000 secs
Ready.
Reading 0 ... 172000  =      0.000 ...   172.000 secs...


C:\Users\marco\AppData\Local\Temp\ipykernel_6904\2599999032.py:1: RuntimeWarning: This filename (../SEED_FRA_OUTPUT/POSITIVE/1_1_trial_0_signals.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg_data = mne.io.read_raw_fif("../SEED_FRA_OUTPUT/POSITIVE/1_1_trial_0_signals.fif", preload=True)


array([[1.82471730e-05, 1.45351186e-05, 1.29381182e-04, 2.61849569e-08],
       [1.73157280e-05, 1.21239518e-05, 9.95016221e-05, 1.39091940e-08],
       [1.76477237e-05, 1.24551339e-05, 1.00954722e-04, 5.33043383e-08],
       [1.54707995e-05, 1.06580271e-05, 9.26110134e-05, 1.89229649e-08],
       [1.50409145e-05, 9.41240397e-06, 6.96754612e-05, 4.49656615e-08],
       [2.12736645e-05, 1.52300618e-05, 1.60581695e-04, 4.62834347e-09],
       [1.63211366e-05, 1.09845523e-05, 1.13915541e-04, 1.06091548e-08],
       [1.30994504e-05, 8.22571946e-06, 7.02182222e-05, 3.08602969e-08],
       [1.27690648e-05, 7.90447103e-06, 6.65669903e-05, 9.22963866e-10],
       [1.11853412e-05, 6.45676690e-06, 5.54241366e-05, 3.96961859e-08],
       [1.25778471e-05, 7.18045646e-06, 5.51310181e-05, 4.56300022e-08],
       [1.37614043e-05, 8.23329311e-06, 6.41239505e-05, 2.74292225e-08],
       [1.61317967e-05, 1.03736411e-05, 9.34687375e-05, 1.45839185e-08],
       [1.96326594e-05, 1.42129393e-05, 1.49159283e

In [102]:
data_model_dl = np.array( [ *get_data_DL("../SEED_FRA_OUTPUT/POSITIVE", 1), *get_data_DL("../SEED_FRA_OUTPUT/NEGATIVE", -1), *get_data_DL("../SEED_FRA_OUTPUT/NEUTRAL", 0) ] )

Opening raw data file ../SEED_FRA_OUTPUT/POSITIVE/1_1_trial_0_signals.fif...
    Range : 21000 ... 193000 =     21.000 ...   193.000 secs
Ready.
Reading 0 ... 172000  =      0.000 ...   172.000 secs...


C:\Users\marco\AppData\Local\Temp\ipykernel_6904\2770616247.py:12: RuntimeWarning: This filename (../SEED_FRA_OUTPUT/POSITIVE/1_1_trial_0_signals.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg_data = mne.io.read_raw_fif(name, preload=True)


Opening raw data file ../SEED_FRA_OUTPUT/POSITIVE/1_1_trial_11_signals.fif...
    Range : 2327000 ... 2471000 =   2327.000 ...  2471.000 secs
Ready.
Reading 0 ... 144000  =      0.000 ...   144.000 secs...


C:\Users\marco\AppData\Local\Temp\ipykernel_6904\2770616247.py:12: RuntimeWarning: This filename (../SEED_FRA_OUTPUT/POSITIVE/1_1_trial_11_signals.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg_data = mne.io.read_raw_fif(name, preload=True)


Opening raw data file ../SEED_FRA_OUTPUT/POSITIVE/1_1_trial_12_signals.fif...
    Range : 2492000 ... 2599000 =   2492.000 ...  2599.000 secs
Ready.
Reading 0 ... 107000  =      0.000 ...   107.000 secs...


...

C:\Users\marco\AppData\Local\Temp\ipykernel_6904\2770616247.py:12: RuntimeWarning: This filename (../SEED_FRA_OUTPUT/NEUTRAL/8_3_trial_2_signals.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg_data = mne.io.read_raw_fif(name, preload=True)


Opening raw data file ../SEED_FRA_OUTPUT/NEUTRAL/8_3_trial_5_signals.fif...
    Range : 1103000 ... 1232000 =   1103.000 ...  1232.000 secs
Ready.
Reading 0 ... 129000  =      0.000 ...   129.000 secs...


C:\Users\marco\AppData\Local\Temp\ipykernel_6904\2770616247.py:12: RuntimeWarning: This filename (../SEED_FRA_OUTPUT/NEUTRAL/8_3_trial_5_signals.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg_data = mne.io.read_raw_fif(name, preload=True)


Opening raw data file ../SEED_FRA_OUTPUT/NEUTRAL/8_3_trial_7_signals.fif...
    Range : 1572000 ... 1718000 =   1572.000 ...  1718.000 secs
Ready.
Reading 0 ... 146000  =      0.000 ...   146.000 secs...


C:\Users\marco\AppData\Local\Temp\ipykernel_6904\2770616247.py:12: RuntimeWarning: This filename (../SEED_FRA_OUTPUT/NEUTRAL/8_3_trial_7_signals.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg_data = mne.io.read_raw_fif(name, preload=True)


In [103]:
#np.save('matrices_etiquetadas.npy', data_model_dl)

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from sklearn.model_selection import train_test_split

In [3]:
X = np.load('matrices_etiquetadas.npy')

In [4]:
y = np.array([*[1 for _ in range(168)], *[-1 for _ in range(168)], *[0 for _ in range(168)]] )

In [5]:
y

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [128]:
X_train = X_train.reshape(X_train.shape[0], 62, 4, 1)
X_test = X_test.reshape(X_test.shape[0], 62, 4, 1)

In [45]:
model = Sequential([
  Flatten(input_shape=(62,4)),
  Dense(403, activation='relu'),
  Dropout(0.4),
  Dense(10, activation='relu'),
  Dense(1, activation="softmax")
])

#Model compilation
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              #loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
X_train.dtype

dtype('float64')

In [35]:
y_train = y_train.astype('float64')

In [46]:
model.fit( X_train, y_train, epochs = 16 )

Epoch 1/16
13/13 [==============================] - 2s 9ms/step - loss: 0.6516 - accuracy: 0.3325
Epoch 2/16
13/13 [==============================] - 0s 8ms/step - loss: 0.5308 - accuracy: 0.3325
Epoch 3/16
13/13 [==============================] - 0s 8ms/step - loss: 0.3858 - accuracy: 0.3325
Epoch 4/16
13/13 [==============================] - 0s 8ms/step - loss: 0.2413 - accuracy: 0.3325
Epoch 5/16
13/13 [==============================] - 0s 8ms/step - loss: 0.1309 - accuracy: 0.3325
Epoch 6/16
13/13 [==============================] - 0s 8ms/step - loss: 0.0560 - accuracy: 0.3325
Epoch 7/16
13/13 [==============================] - 0s 8ms/step - loss: 0.0405 - accuracy: 0.3325
Epoch 8/16
13/13 [==============================] - 0s 9ms/step - loss: 0.0155 - accuracy: 0.3325
Epoch 9/16
13/13 [==============================] - 0s 8ms/step - loss: -0.0038 - accuracy: 0.3325
Epoch 10/16
13/13 [==============================] - 0s 8ms/step - loss: -0.0155 - accuracy: 0.3325
Epoch 11/16
13/13

In [7]:
### LAST TRY

In [10]:
# Definición del modelo
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(62, 4, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(MaxPooling2D(pool_size=(2, 2))) # Capa de agrupación adicional
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))



ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv2d_5. Consider increasing the input size. Received input shape [None, 30, 1, 32] which would produce output shape with a zero or negative value in a dimension.